## Gerekli kütüphaneler:

In [1]:
import sys

sys.path.append("../..")

import os
import cv2
import numpy as np
import joblib
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from utils.ml_features.preprocessing import extract_features, apply_pca
from utils.ml_features.grid_search import grid_search

## Veri yolu ve etiketler:

In [2]:
data_dir = "../../data/uc_merced"
class_names = sorted(os.listdir(data_dir))

## Feature çıkarma:

In [3]:
x_data, y_data = [], []

for label, cls in enumerate(class_names):
    cls_path = os.path.join(data_dir, cls)
    print("İşlenen class:", cls)
    for fname in tqdm(os.listdir(cls_path), desc=f"{cls}", leave=False):
        img_path = os.path.join(cls_path, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        features = extract_features(img)
        x_data.append(features)
        y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)
print("Toplam örnek:", len(x_data), "| Feature boyutu:",  x_data.shape[1])

İşlenen class: agricultural


İşlenen class: airplane


İşlenen class: baseballdiamond


İşlenen class: beach


İşlenen class: buildings


...

İşlenen class: runway


İşlenen class: sparseresidential


İşlenen class: storagetanks


İşlenen class: tenniscourt


Toplam örnek: 2100 | Feature boyutu: 1803


## PCA kullanarak boyut azaltımı:

In [4]:
x_reduced = apply_pca(False, x_data, n_components=100)

## Veriyi bölme:

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y_data, test_size=0.2, stratify=y_data, random_state=42)

## Grid search ile random forest:

In [6]:
model = RandomForestClassifier()
params = {
    "n_estimators": [75, 100],
    "max_depth": [10, 20]
}

print("Random Forest eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_rf.pkl")
print("Random Forest için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Random Forest eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 4


RandomForestClassifier Grid Search:  25%|██▌       | 1/4 [00:08<00:25,  8.60s/it]

Parametreler: {'n_estimators': 75, 'max_depth': 10} | Skor: 0.6869047619047619 | Süre: 8 sn


RandomForestClassifier Grid Search:  50%|█████     | 2/4 [00:17<00:17,  8.61s/it]

Parametreler: {'n_estimators': 75, 'max_depth': 20} | Skor: 0.724404761904762 | Süre: 8 sn


RandomForestClassifier Grid Search:  75%|███████▌  | 3/4 [00:20<00:06,  6.13s/it]

Parametreler: {'n_estimators': 100, 'max_depth': 10} | Skor: 0.6988095238095238 | Süre: 3 sn


RandomForestClassifier Grid Search: 100%|██████████| 4/4 [00:24<00:00,  6.09s/it]

Parametreler: {'n_estimators': 100, 'max_depth': 20} | Skor: 0.7375 | Süre: 3 sn


Random Forest için en iyi skor: 0.7375 | Parametreler: {'n_estimators': 100, 'max_depth': 20} | Toplam Süre: 24 sn 



## Grid search ile gradient boosting:

In [7]:
model = GradientBoostingClassifier()
params = {
    "n_estimators": [50]
}

print("Gradient Boosting eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_gb.pkl")
print("Gradient Boosting için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Gradient Boosting eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 1


GradientBoostingClassifier Grid Search: 100%|██████████| 1/1 [01:43<00:00, 103.53s/it]

Parametreler: {'n_estimators': 50} | Skor: 0.6261904761904763 | Süre: 1 dk 43 sn


Gradient Boosting için en iyi skor: 0.6261904761904763 | Parametreler: {'n_estimators': 50} | Toplam Süre: 1 dk 43 sn 



## Grid search ile multilayer perceptron:

In [8]:
model = MLPClassifier(max_iter=300)
params = {
    "hidden_layer_sizes": [(100,), (128,)],
    "alpha": [0.0001],
    "learning_rate_init": [0.001]
}

print("Multilayer Perceptron eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_mlp.pkl")
print("Multilayer Perceptron için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Multilayer Perceptron eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 2


MLPClassifier Grid Search:  50%|█████     | 1/2 [00:03<00:03,  3.66s/it]

Parametreler: {'hidden_layer_sizes': (100,), 'alpha': 0.0001, 'learning_rate_init': 0.001} | Skor: 0.5190476190476191 | Süre: 3 sn


MLPClassifier Grid Search: 100%|██████████| 2/2 [00:06<00:00,  3.49s/it]

Parametreler: {'hidden_layer_sizes': (128,), 'alpha': 0.0001, 'learning_rate_init': 0.001} | Skor: 0.5029761904761905 | Süre: 3 sn


Multilayer Perceptron için en iyi skor: 0.5190476190476191 | Parametreler: {'hidden_layer_sizes': (100,), 'alpha': 0.0001, 'learning_rate_init': 0.001} | Toplam Süre: 6 sn 

